## Radio interferometría y síntesis de imágenes en astronomía - Laboratorio 2

### Vicente Mieres

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# imports
import numpy as np
# modules
from modules.simulation import visibilities_simulation

In [3]:
# simulate visibilities no grid
VLA_L_4 = {
  "latitude": 34.078749,
  "longitude": -107.617728,
  "file_route": "../antenna_arrays/alma.cycle10.8.cfg",
  "catalog_source": "Sirius",
  "utc_start": "2024-10-21T00:00:00",
  "utc_end": "2024-10-21T06:00:00",
  "step_min": 5,
  "n_freqs": 4,
  "interferometer": {
    "name": "VLA",
    "band_name": "L" },
  "n_sources": 100,
  "max_offset_deg": 1.0,
  "flux_range": [0.1, 2.0],
  "seed": 42
  }

V, uvw_lambda, frequencies, baselines_enu = visibilities_simulation(VLA_L_4)

In [4]:
print(V.shape)

(903, 73, 4)


In [5]:
from modules.coords import max_basline
from modules.interferometry import grid_visibilities
import cupy as cp

# Resolucion
N = 4096
# Distancion maxima entre baselines
Dmax = max_basline(baselines_enu)
oversampling_factor = 10

c = 299792458.0 
freq = np.min(frequencies)
min_wavelenghgt = c / freq
dx = dy = (min_wavelenghgt / Dmax) / oversampling_factor

imgs = []
dus = []

du = 1.0 / (N * dx)
dv = du

In [6]:
import cupy as cp
print("CuPy CUDA runtime version:", cp.cuda.runtime.runtimeGetVersion())


CuPy CUDA runtime version: 12090


In [ ]:
%%time
# CPU
VG2, WG2 = grid_visibilities(V, uvw_lambda, du, dv, N, use_gpu=False)

Gridding in CPU (NumPy Vectorized)
CPU times: total: 46.9 ms
Wall time: 38.6 ms


In [20]:
%%time
# GPU
VG, WG  = grid_visibilities(V, uvw_lambda, du, dv, Npix=N)

Gridding in GPU (CuPy Vectorized)
CPU times: total: 15.6 ms
Wall time: 7 ms


Python: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
CuPy: 13.6.0
CuPy runtime version: 12090
CuPy device count: 1
CuPy current device: 0
Numba: 0.62.1
Numba driver version: (13, 0)
Numba/driver check error: exception: access violation reading 0xFFFFFFFFFFFFFFFF
